## CriteriaGroup is another (preferred) way to specify AND and OR conditions in the query: Fetch the female patient cohort with Low back pain OR Type 2 diabetes mellitus

Setup the imports.

In [ ]:
from pyqe import *

Always start with creating a query object. 

Apart from creating Query object, it does the following that are required for setting up your profile:
- Ask for username and password for login

In [ ]:
# Always begin your script by creating Query object
query_female_patients = Query("female_patients_with_low_backpain_or_type2_diabetes_mellitus")

Use get_study_list() method to fetch all available studies that you have access to.

Use set_study() method to select a study that you are interested in (by passing the study ID).

In [ ]:
query_female_patients.get_study_list()

# any STUDY_ID from above list
query_female_patients.set_study('703c5d8a-a1d9-4d42-a314-5b9aad513390') 

Similar to above step, use get_config_list() method to fetch all available configurations corresponding to the study.

Use set_study_config() method to select a config (by passing the config ID).

In [ ]:
query_female_patients.get_config_list()

# any CONFIG_ID from above list
query_female_patients.set_study_config('088791fd-cac7-4281-9d61-6f4efd92d2f5') 

Create two Expression objects as described below:
- ComparisonOperator.EQUAL (strictly match the value) and 'Low back pain' (a value that our expression relies on)
- ComparisonOperator.EQUAL (strictly match the value) and 'Type2 Diabetes Mellitus' (a value that our expression relies on)

In [ ]:
exp_equal_low_backpain = Expression(ComparisonOperator.EQUAL, 'Low back pain')
exp_equal_type2_diabetes_mellitus = Expression(ComparisonOperator.EQUAL, 'Type 2 diabetes mellitus')

Create a low backpain condition constraint:
- Create a Constraint object
- Add the exp_equal_low_backpain to the constraint

In [ ]:
constraint_low_back_pain = Constraint()
constraint_low_back_pain.add(exp_equal_low_backpain)

Create a type 2 diabetes condition constraint:
- Create a Constraint object
- Add the exp_equal_type2_diabetes_mellitus to the constraint

In [ ]:
constraint_type2_diabetes_mellitus = Constraint()
constraint_type2_diabetes_mellitus.add(exp_equal_type2_diabetes_mellitus)

Use Interactions class to create a filtercard of type ConditionOccurrence with name 'Type2 Diabetes Mellitus'.

NOTE: Interactions is a parent class which can help to define/instantiate any filter card that is available in the study configuration. 

Go to Miscellaneous to find out more on how to view all filter cards.

Call add_condition_name() method to add the constraint constraint_type2_diabetes_mellitus

In [ ]:
## 6.1 Create two filtercards, filtercard_diabetes_type2 and filtercard_low_backpain
filtercard_diabetes_type2 = Interactions.ConditionOccurrence('Type2 Diabetes Mellitus')
filtercard_diabetes_type2.add_condition_name([constraint_type2_diabetes_mellitus])

Use Interactions class to create a filtercard of type ConditionOccurrence with name 'Low backpain'.

Call add_condition_name() method to add the constraint constraint_low_back_pain.

In [ ]:
filtercard_low_backpain = Interactions.ConditionOccurrence('Low backpain')
filtercard_low_backpain.add_condition_name([constraint_low_back_pain])

Create CriteriaGroup object criteriagroup_type2_diabetes_or_low_backpain with the following values:
- MatchCriteria.ANY condition (ANY of the filter cards' conditions added to the criteriagroup must hold true)
- List of filter cards that the MatchCriteria condition must be applied on

In [ ]:
## 6.2 create CriteriaGroup with OR condition on above filtercards
criteriagroup_type2_diabetes_or_low_backpain = CriteriaGroup(MatchCriteria.ANY, [filtercard_diabetes_type2, filtercard_low_backpain])


Create a gender constraint:
- Create a Constraint object
- Create and add an expression (that EQUALS 'Female') to the Constraint object

In [ ]:
## 6.3 create Patient filtercard with female constraint
constraint_female_gender = Constraint()
constraint_female_gender.add(Expression(ComparisonOperator.EQUAL, 'Female'))

Create a filter card:
- Create a filtercard of type Patient class
- Call add_gender() to add the constraint constraint_female_gender

NOTE: Patient object has multiple methods in order to add constraints, for example, add_county(), add_race(), add_state(). 

Go to Miscellaneous to find out more on how to view all such methods.

In [ ]:
filtercard_female_patient = Person.Patient()
filtercard_female_patient.add_gender([constraint_female_gender])

Create CriteriaGroup object criteriagroup_female_patient with the following values:
- MatchCriteria.ALL condition (ALL of the filter cards' conditions added to the CriteriaGroup must hold true)
- List of filter cards that the MatchCriteria condition must be applied on

In [ ]:
criteriagroup_female_patient = CriteriaGroup(MatchCriteria.ALL, [filtercard_female_patient])

Add criteriagroup_type2_diabetes_or_low_backpain to the object criteriagroup_female_patient.

NOTE: It means all filter cards that exist in criteriagroup_type2_diabetes_or_low_backpain will also be applied to the Result.

In [ ]:
## 6.4 perform AND operation between two CriteriaGroups
criteriagroup_female_patient.add_exclusive_group(criteriagroup_type2_diabetes_or_low_backpain)

Add criteriagroup_female_patient to the query created in step 1.

In [ ]:
# 6.5 add CriteriaGroup to the query object and fetch the cohort
query_female_patients.add_criteria_group(criteriagroup_female_patient)

Use the query object, call the method get_cohort() to create a cohort object preparing for fetching dataframe. Please select the required entity when asked and rerun this step to select a different entity for a dataframe.

In [ ]:
request_female_patients_with_type2_diabetes_or_low_backpain = query.get_dataframe_cohort()

Further create a Result class object (that does magic).

In our case, download_dataframe() method is called by passing the cohort object created above. 

Returned result will be a list of patients matching (strictly) both filter cards created above

In [ ]:
female_patients_with_low_backpain_or_type2_diabetes_mellitus = Result().download_dataframe(request_female_patients_with_type2_diabetes_or_low_backpain)
print(f'\n Participants: {female_patients_with_low_backpain_or_type2_diabetes_mellitus.head(10)}')